## Task 3 - Sentiment Analysis
Use google BERT word embedding model to build a sentiment analysis model.


#### Assumptions
* Emojis would not be removed since they might be very important for discriminating sentiment in text.
* URLs, Hashtags, Mentions, repeating dots, repeating characters than occur more than twice would be removed.
* Arabic Diacritics would be removed.
* Due to class imbalance, __balanced class weighting__ would be used.


#### Datasets collected and used
* __TEAD sample data__: (392,487 positive and 163,436 negative cleaned arabic tweets)
    * [GitHub](https://github.com/HSMAabdellaoui/TEAD)
    * Sample data [Google Drive](https://drive.google.com/drive/folders/1pfqK21nEKL8xYt3L4e09tMIQSnI_5G-7?usp=sharing)
* __LABR dataset__: 63,000 book reviews in arabic
    * [GitHub](https://github.com/mohamedadaly/LABR)
* __ArSAS dataset__: 21,000 arabic tweets
    * [Website](https://homepages.inf.ed.ac.uk/wmagdy/resources.htm)
* __AJGT dataset__: 1800 Jordanian tweets
    * [GitHub](https://github.com/komari6/Arabic-twitter-corpus-AJGT)
* __40000 Egyptian tweet dataset__
    * [Website](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/LBXV9O)
* __SemEval2017-task4 subtask-A arabic dataset__: 3355 arabic tweets
    * [Website](https://alt.qcri.org/semeval2017/task4/index.php?id=data-and-tools)
* __ASTD dataset__: 10,000 arabic tweets
    * [GitHub](https://github.com/mahmoudnabil/ASTD)

__Note__: The pre-trained CAMeL-Lab Sentiment Classifier BERT [model](https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment) which used the [bert-base-arabic-camelbert-da](https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-da) language model was trained on the __ArSAS__, __ASTD__, and __SemEval__ datasets. 

#### Labels
* __0__: Negative Sentiment
* __1__: Neutral Sentiment
* __2__: Positive Sentiment


####  Combined dataset
* __Total records__: 693,917 records
    * __Positive__: 462,133 records
    * __Neutral__: 29,059 records
    * __Negative__: 202,725 records

In [1]:
import pandas as pd
import numpy as np
import re
import pyarabic.araby as araby  # for removing diacritics
import torch
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight  # For balanced class weighting training
from camel_tools.sentiment import SentimentAnalyzer  # For testing the pre-trained BERT classifier model
from transformers import AutoTokenizer, AutoModelForSequenceClassification  # Training sentiment classification model
from datasets import Dataset

/home/tamer/anaconda3/envs/psut-nlp-project/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# From Tasks 1 and 2
def clean(sentence):
    def remove_diacritics(string):
        return araby.strip_diacritics(string)
    
    re_general_pattern = r"https?:\/\/.*[\r\n]*|#\w+|@\w+|\.{2,}"
    re_repeating_character_pattern = r"(\w)\1{2,}"

    # 1- Removing URLs, Hashtags, Mentions, and repeating dots
    sentence = re.sub(re_general_pattern, "", sentence)
    # 2- Removing repeating characters that occur more than twice
    sentence = re.sub(re_repeating_character_pattern, r"\1", sentence)
    # 3- Removing Arabic Diacritics
    sentence = remove_diacritics(sentence)

    return sentence

### 1- Testing the pre-trained CAMeL-Lab Arabic sentiment classifier BERT model

In [3]:
camel_bert_senti_model = SentimentAnalyzer.pretrained()

In [4]:
dir(camel_bert_senti_model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'labels',
 'labels_map',
 'model',
 'predict',
 'predict_sentence',
 'pretrained',
 'tokenizer',
 'use_gpu']

In [5]:
camel_bert_senti_model.use_gpu

True

In [6]:
# Predict the sentiment of a single sentence
sentiment = camel_bert_senti_model.predict_sentence('أنا بخير')

# Predict the sentiment of multiple sentences
sentences = [
    'أنا بخير',
    'أنا لست بخير'
]
sentiments = camel_bert_senti_model.predict(sentences)

In [7]:
sentiments

['positive', 'negative']

#### 1.1 Testing on first 10 tweets from file1.txt

In [8]:
df = pd.read_csv("data/file1.txt", sep="\t").drop("tweetID", axis=1)

In [9]:
for index, tweet in df.iterrows():
    if index == 10:
        break
        
    sentence = clean(tweet[0])
    pred = camel_bert_senti_model.predict_sentence(sentence)
    print("---------")
    print(f"Tweet {index}\n")
    print(f"Original Tweet: {tweet[0]}\n")
    print(f"Prediction: {pred}")

---------
Tweet 0

Original Tweet: الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn

Prediction: neutral
---------
Tweet 1

Original Tweet: #الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR

Prediction: neutral
---------
Tweet 2

Original Tweet: @helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري

Prediction: negative
---------
Tweet 3

Original Tweet: #النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu

Prediction: negative
---------
Tweet 4

Original Tweet: البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh

Prediction: neutral
---------
Tweet 5

Original Tweet: @El__DoN__ABoOoD @soliman_sport توهير تفكيره يورو على يورو يطلع ٢ يورو وعلى كذا ثقافة آسيوية م

### 2- Preparing the Arabic sentiment analysis dataset

#### 2.1 TEAD sample data

##### 2.1.1 Loading positive sentences

In [10]:
tead_pos_df = pd.read_csv("data/sentiment-analysis/TEAD_sample_data_final_positive_clean.txt", header=None, names=["sentence"])

In [11]:
tead_pos_df

,sentence
0,مراا حلو ان الاستاذه قايله برسل لكم التخطيط وت...
1,نلقى أي قروب يحكي علي ال ندير انسحاب تكتيكي لا...
2,يارب شعور إني أكون مروقه ومبسوطه يكون كل يوم
3,حاجات تزيد الانثي انوثه تتروش مع رجالها ولا تس...
4,وضع الانترنت واجد يضايق لا برودباند البيت ولا ...
...,...
392482,عاد مارسييلو إلي نعرفه دفاع هجوم إبداع إمتاع إ...
392483,قلت أحبك وقلبي قال بين المحاني والله إن كلمة أ...
392484,انا وحنان بنتكلم واتس وماسنجر و ف نفس الوقت وف...
392485,الآ بذكر الله تطمئن القلؤب غرد الان بذكر الله


In [12]:
tead_pos_df["label"] = 2

In [13]:
tead_pos_df

,sentence,label
0,مراا حلو ان الاستاذه قايله برسل لكم التخطيط وت...,2
1,نلقى أي قروب يحكي علي ال ندير انسحاب تكتيكي لا...,2
2,يارب شعور إني أكون مروقه ومبسوطه يكون كل يوم,2
3,حاجات تزيد الانثي انوثه تتروش مع رجالها ولا تس...,2
4,وضع الانترنت واجد يضايق لا برودباند البيت ولا ...,2
...,...,...
392482,عاد مارسييلو إلي نعرفه دفاع هجوم إبداع إمتاع إ...,2
392483,قلت أحبك وقلبي قال بين المحاني والله إن كلمة أ...,2
392484,انا وحنان بنتكلم واتس وماسنجر و ف نفس الوقت وف...,2
392485,الآ بذكر الله تطمئن القلؤب غرد الان بذكر الله,2


##### 2.1.2 Loading negative sentences

In [14]:
tead_neg_df = pd.read_csv("data/sentiment-analysis/TEAD_sample_data_final_negative_clean.txt", header=None, names=["sentence"])

In [15]:
tead_neg_df

,sentence
0,السعوديين شافو ترمب و مومصدقين نفسهم لعمى تقلو...
1,شكرا ويستهام لك كل الاحترام
2,تضعك المعرفة في صفوف الحكماء و يضعك العمل في ص...
3,خالي الف رحمةة ع روحةة
4,بيريز جيب بونوتشي يالذيب مانبي هجوم
...,...
163431,وش حاجتكك بقلبي يلي خذيته كلي ولهه
163432,لكل شيء لابد نهاية و هذي النهاية ي تشابي
163433,كنت عشمانة في ليله حلوة بس الظروف ليها رأي اخر
163434,البندر قبل قال بيعرض صوت ام حمود وهي متوفيه اصلن


In [16]:
tead_neg_df["label"] = 0

In [17]:
tead_neg_df

,sentence,label
0,السعوديين شافو ترمب و مومصدقين نفسهم لعمى تقلو...,0
1,شكرا ويستهام لك كل الاحترام,0
2,تضعك المعرفة في صفوف الحكماء و يضعك العمل في ص...,0
3,خالي الف رحمةة ع روحةة,0
4,بيريز جيب بونوتشي يالذيب مانبي هجوم,0
...,...,...
163431,وش حاجتكك بقلبي يلي خذيته كلي ولهه,0
163432,لكل شيء لابد نهاية و هذي النهاية ي تشابي,0
163433,كنت عشمانة في ليله حلوة بس الظروف ليها رأي اخر,0
163434,البندر قبل قال بيعرض صوت ام حمود وهي متوفيه اصلن,0


#### 2.2 LABR dataset

* 1 and 2 star ratings will be considered as "Negative"
* 3 star ratings will be considered as "Neutral"
* 4 and 5 star ratings will be considered as "Positive"

In [18]:
labr_df = pd.read_csv("data/sentiment-analysis/LABR.tsv", sep="\t", header=None, names=["label", "id1", "id2", "id3", "sentence"])

In [19]:
labr_df

,label,id1,id2,id3,sentence
0,4,338670838,7878381,13431841,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني..."
1,4,39428407,1775679,3554772,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...
2,4,32159373,1304410,3554772,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
3,1,442326656,11333112,3554772,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...
4,5,46492258,580165,3554772,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...
...,...,...,...,...,...
63252,5,108789247,3926131,1420,اجمل مسرحية ألفت في تاريخ الادب الانجليزي
63253,3,513749112,16666895,7286365,بصراحة، لم تكن هذه الرواية على قدر توقعاتي. ا...
63254,3,137280339,4303773,7286365,هي الرواية الأولى التي قرأتها لأيميلي نصر الل...
63255,3,175939769,4770537,7286365,تَدْخل بيوت الناس، تدخل قلوبهم، تمدّ يدك تصاف...


In [20]:
labr_df.drop(["id1", "id2", "id3"], axis=1, inplace=True)

In [21]:
labr_df

,label,sentence
0,4,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني..."
1,4,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...
2,4,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
3,1,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...
4,5,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...
...,...,...
63252,5,اجمل مسرحية ألفت في تاريخ الادب الانجليزي
63253,3,بصراحة، لم تكن هذه الرواية على قدر توقعاتي. ا...
63254,3,هي الرواية الأولى التي قرأتها لأيميلي نصر الل...
63255,3,تَدْخل بيوت الناس، تدخل قلوبهم، تمدّ يدك تصاف...


In [22]:
labr_df = labr_df.reindex(['sentence','label'], axis=1)

In [23]:
labr_df

,sentence,label
0,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني...",4
1,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...,4
2,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...,4
3,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...,1
4,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...,5
...,...,...
63252,اجمل مسرحية ألفت في تاريخ الادب الانجليزي,5
63253,بصراحة، لم تكن هذه الرواية على قدر توقعاتي. ا...,3
63254,هي الرواية الأولى التي قرأتها لأيميلي نصر الل...,3
63255,تَدْخل بيوت الناس، تدخل قلوبهم، تمدّ يدك تصاف...,3


In [24]:
labr_df['label'].loc[(labr_df['label'] < 3)] = 0

/home/tamer/anaconda3/envs/psut-nlp-project/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [25]:
labr_df['label'].loc[(labr_df['label'] == 3)] = 1

In [26]:
labr_df['label'].loc[(labr_df['label'] > 3)] = 2

In [27]:
labr_df

,sentence,label
0,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني...",2
1,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...,2
2,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...,2
3,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...,0
4,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...,2
...,...,...
63252,اجمل مسرحية ألفت في تاريخ الادب الانجليزي,2
63253,بصراحة، لم تكن هذه الرواية على قدر توقعاتي. ا...,1
63254,هي الرواية الأولى التي قرأتها لأيميلي نصر الل...,1
63255,تَدْخل بيوت الناس، تدخل قلوبهم، تمدّ يدك تصاف...,1


In [28]:
labr_df["label"].value_counts()

2    42832
1    12201
0     8224
Name: label, dtype: int64

#### 2.3 ArSAS dataset

Assuming "Mixed" label as "Neutral".

In [29]:
arsas_df = pd.read_csv("data/sentiment-analysis/ArSAS.txt", sep="\t")

In [30]:
arsas_df

,#Tweet_ID,Tweet_text,Topic,Sentiment_label,Sentiment_label_confidence,Speech_act_label,Speech_act_label_confidence
0,929241870508724224,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,Event,Positive,0.38,Assertion,0.62
1,928942264583376897,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,Entity,Negative,1.00,Expression,0.68
2,928615163250520065,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,Event,Positive,0.69,Assertion,1.00
3,931614713368186880,ومع السيسي و بشار و ايران و بن زايد و والا خلي...,Event,Negative,1.00,Expression,1.00
4,929755693011427331,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 20...,Event,Neutral,1.00,Assertion,1.00
...,...,...,...,...,...,...,...
19892,929109605267000960,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 ال...,Event,Neutral,1.00,Assertion,1.00
19893,930131859039837952,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز...,Event,Positive,0.69,Expression,0.69
19894,928739551136361984,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة ا...,Event,Neutral,1.00,Assertion,1.00
19895,929254320754823168,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله ...,Event,Neutral,1.00,Assertion,1.00


In [31]:
arsas_df.drop(["#Tweet_ID", "Topic", "Sentiment_label_confidence", "Speech_act_label", "Speech_act_label_confidence"], axis=1, inplace=True)

In [32]:
arsas_df

,Tweet_text,Sentiment_label
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,Positive
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,Negative
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,Positive
3,ومع السيسي و بشار و ايران و بن زايد و والا خلي...,Negative
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 20...,Neutral
...,...,...
19892,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 ال...,Neutral
19893,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز...,Positive
19894,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة ا...,Neutral
19895,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله ...,Neutral


In [33]:
arsas_df.rename(columns={'Tweet_text': 'sentence', 'Sentiment_label': 'label'}, inplace=True)

In [34]:
arsas_df

,sentence,label
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,Positive
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,Negative
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,Positive
3,ومع السيسي و بشار و ايران و بن زايد و والا خلي...,Negative
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 20...,Neutral
...,...,...
19892,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 ال...,Neutral
19893,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز...,Positive
19894,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة ا...,Neutral
19895,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله ...,Neutral


In [35]:
arsas_df['label'].loc[(arsas_df['label'] == "Negative")] = 0

In [36]:
arsas_df['label'].loc[(arsas_df['label'] == "Neutral")] = 1

In [37]:
arsas_df['label'].loc[(arsas_df['label'] == "Mixed")] = 1

In [38]:
arsas_df['label'].loc[(arsas_df['label'] == "Positive")] = 2

In [39]:
arsas_df

,sentence,label
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,2
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,0
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,2
3,ومع السيسي و بشار و ايران و بن زايد و والا خلي...,0
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 20...,1
...,...,...
19892,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 ال...,1
19893,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز...,2
19894,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة ا...,1
19895,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله ...,1


In [40]:
arsas_df["label"].value_counts()

1    8113
0    7384
2    4400
Name: label, dtype: int64

#### 2.4 AJGT dataset

In [41]:
ajgt_df = pd.read_excel("data/sentiment-analysis/AJGT.xlsx")

In [42]:
ajgt_df

,ID,Feed,Sentiment
0,1,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,Positive
1,2,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,Negative
2,3,كله رائع بجد ربنا يكرمك,Positive
3,4,لسانك قذر يا قمامه,Negative
4,5,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,Negative
...,...,...,...
1795,1796,يلعن شرف خواتك,Negative
1796,1797,يلعنك نذل وكلب,Negative
1797,1798,يمتاز الاردن بتنوع السياحه فيه فتنقسم السياحه ...,Positive
1798,1799,ينور عليك ويكثر من امثالك,Positive


In [43]:
ajgt_df.drop(["ID"], axis=1, inplace=True)

In [44]:
ajgt_df.rename(columns={"Feed": "sentence", "Sentiment": "label"}, inplace=True)

In [45]:
ajgt_df

,sentence,label
0,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,Positive
1,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,Negative
2,كله رائع بجد ربنا يكرمك,Positive
3,لسانك قذر يا قمامه,Negative
4,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,Negative
...,...,...
1795,يلعن شرف خواتك,Negative
1796,يلعنك نذل وكلب,Negative
1797,يمتاز الاردن بتنوع السياحه فيه فتنقسم السياحه ...,Positive
1798,ينور عليك ويكثر من امثالك,Positive


In [46]:
ajgt_df['label'].loc[(ajgt_df['label'] == "Negative")] = 0

In [47]:
ajgt_df['label'].loc[(ajgt_df['label'] == "Positive")] = 2

In [48]:
ajgt_df

,sentence,label
0,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,2
1,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,0
2,كله رائع بجد ربنا يكرمك,2
3,لسانك قذر يا قمامه,0
4,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,0
...,...,...
1795,يلعن شرف خواتك,0
1796,يلعنك نذل وكلب,0
1797,يمتاز الاردن بتنوع السياحه فيه فتنقسم السياحه ...,2
1798,ينور عليك ويكثر من امثالك,2


In [49]:
ajgt_df["label"].value_counts()

2    900
0    900
Name: label, dtype: int64

#### 2.5 40000 Egyptian tweets dataset

In [50]:
egypt_df = pd.read_excel("data/sentiment-analysis/40000-Egyptian-tweets.xlsx")

In [51]:
egypt_df

,review,label
0,اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش ...,negative
1,دائما اكره اخر ليله في كل مكان .,negative
2,يارب اللى يسرق تويتاتى يدخل النار .,negative
3,الاسراف فى تناول القهوة يسبب الوفاه .,negative
4,انا اتبهدلت من التراب النهارده. حاجة تقرف .,positive
...,...,...
39995,لنسعد ايامنا بالابتسامة بدلاً ان نملاها بالدموع.,positive
39996,مش هقولك غير ان نص الضحك اللي ضحكته فى حياتي ك...,positive
39997,ربنا يوفقك ويسهلك وان شاء الله تعدي الفترة دي ...,positive
39998,مبسوطة اوى عملت طريقة مكرونة جديدة و هى دلوقتى...,positive


In [52]:
egypt_df.rename(columns={"review": "sentence"}, inplace=True)

In [53]:
egypt_df['label'].loc[(egypt_df['label'] == "negative")] = 0

In [54]:
egypt_df['label'].loc[(egypt_df['label'] == "positive")] = 2

In [55]:
egypt_df

,sentence,label
0,اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش ...,0
1,دائما اكره اخر ليله في كل مكان .,0
2,يارب اللى يسرق تويتاتى يدخل النار .,0
3,الاسراف فى تناول القهوة يسبب الوفاه .,0
4,انا اتبهدلت من التراب النهارده. حاجة تقرف .,2
...,...,...
39995,لنسعد ايامنا بالابتسامة بدلاً ان نملاها بالدموع.,2
39996,مش هقولك غير ان نص الضحك اللي ضحكته فى حياتي ك...,2
39997,ربنا يوفقك ويسهلك وان شاء الله تعدي الفترة دي ...,2
39998,مبسوطة اوى عملت طريقة مكرونة جديدة و هى دلوقتى...,2


In [56]:
egypt_df["label"].value_counts()

0                   19998
2                   19995
positive ‎              2
positive ‎              1
positive​               1
positive​‏              1
negative​‏​‏            1
negative​‏​‏​‏​         1
Name: label, dtype: int64

In [57]:
egypt_df["label"].unique()

array([0, 2, 'positive \u200e    ', 'positive\u200b', 'positive \u200e',
       'positive\u200b\u200f', 'negative\u200b\u200f\u200b\u200f',
       'negative\u200b\u200f\u200b\u200f\u200b\u200f\u200b '],
      dtype=object)

In [58]:
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'positive \u200e    '], inplace=True)
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'positive\u200b'], inplace=True)
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'positive \u200e'], inplace=True)
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'positive\u200b\u200f'], inplace=True)
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'negative\u200b\u200f\u200b\u200f'], inplace=True)
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'negative\u200b\u200f\u200b\u200f\u200b\u200f\u200b '], inplace=True)

In [59]:
egypt_df

,sentence,label
0,اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش ...,0
1,دائما اكره اخر ليله في كل مكان .,0
2,يارب اللى يسرق تويتاتى يدخل النار .,0
3,الاسراف فى تناول القهوة يسبب الوفاه .,0
4,انا اتبهدلت من التراب النهارده. حاجة تقرف .,2
...,...,...
39995,لنسعد ايامنا بالابتسامة بدلاً ان نملاها بالدموع.,2
39996,مش هقولك غير ان نص الضحك اللي ضحكته فى حياتي ك...,2
39997,ربنا يوفقك ويسهلك وان شاء الله تعدي الفترة دي ...,2
39998,مبسوطة اوى عملت طريقة مكرونة جديدة و هى دلوقتى...,2


In [60]:
egypt_df["label"].value_counts()

0    19998
2    19995
Name: label, dtype: int64

#### 2.6 SemEval2017-task4 subtask-A arabic dataset

In [61]:
semeval_df = pd.read_csv("data/sentiment-analysis/SemEval2017-task4-train.subtask-A.arabic.txt", sep="\t", header=None, names=["id", "label", "sentence"])

In [62]:
semeval_df

,id,label,sentence
0,783555835494592513,positive,إجبار أبل على التعاون على فك شفرة اجهزتها http...
1,783582397166125056,positive,RT @20fourMedia: #غوغل تتحدى أبل وأمازون بأجهز...
2,783592390728769536,positive,جوجل تنافس أبل وسامسونج بهاتف جديد https://t.c...
3,783597390070685696,positive,رئيس شركة أبل: الواقع المعزز سيصبح أهم من الطع...
4,783617442031472640,neutral,ساعة أبل في الأسواق مرة أخرى https://t.co/dY2x...
...,...,...,...
3348,785923681432117248,positive,طريقة تعطيل اي زر في لوحة المفاتيح على نظام وي...
3349,785924823860076544,neutral,RT @syst__em: إطلاق تحديث تراكمي برقم 14393.32...
3350,785929366756655104,neutral,4 طرق لحذف الملفات المستعصية من الحذف في ويندو...
3351,785951682467246080,neutral,شرح فيديو مهم جدا حل لمشاكل الكومبيوتر كيف تقو...


In [63]:
semeval_df.drop(["id"], axis=1, inplace=True)

In [64]:
semeval_df = semeval_df.reindex(['sentence','label'], axis=1)

In [65]:
semeval_df

,sentence,label
0,إجبار أبل على التعاون على فك شفرة اجهزتها http...,positive
1,RT @20fourMedia: #غوغل تتحدى أبل وأمازون بأجهز...,positive
2,جوجل تنافس أبل وسامسونج بهاتف جديد https://t.c...,positive
3,رئيس شركة أبل: الواقع المعزز سيصبح أهم من الطع...,positive
4,ساعة أبل في الأسواق مرة أخرى https://t.co/dY2x...,neutral
...,...,...
3348,طريقة تعطيل اي زر في لوحة المفاتيح على نظام وي...,positive
3349,RT @syst__em: إطلاق تحديث تراكمي برقم 14393.32...,neutral
3350,4 طرق لحذف الملفات المستعصية من الحذف في ويندو...,neutral
3351,شرح فيديو مهم جدا حل لمشاكل الكومبيوتر كيف تقو...,neutral


In [66]:
semeval_df['label'].loc[(semeval_df['label'] == "negative")] = 0

In [67]:
semeval_df['label'].loc[(semeval_df['label'] == "neutral")] = 1

In [68]:
semeval_df['label'].loc[(semeval_df['label'] == "positive")] = 2

In [69]:
semeval_df

,sentence,label
0,إجبار أبل على التعاون على فك شفرة اجهزتها http...,2
1,RT @20fourMedia: #غوغل تتحدى أبل وأمازون بأجهز...,2
2,جوجل تنافس أبل وسامسونج بهاتف جديد https://t.c...,2
3,رئيس شركة أبل: الواقع المعزز سيصبح أهم من الطع...,2
4,ساعة أبل في الأسواق مرة أخرى https://t.co/dY2x...,1
...,...,...
3348,طريقة تعطيل اي زر في لوحة المفاتيح على نظام وي...,2
3349,RT @syst__em: إطلاق تحديث تراكمي برقم 14393.32...,1
3350,4 طرق لحذف الملفات المستعصية من الحذف في ويندو...,1
3351,شرح فيديو مهم جدا حل لمشاكل الكومبيوتر كيف تقو...,1


In [70]:
semeval_df["label"].value_counts()

1    1470
0    1141
2     742
Name: label, dtype: int64

#### 2.7 ASTD dataset

Assuming "Objective" label as "Neutral" label.

In [71]:
astd_df = pd.read_csv("data/sentiment-analysis/ASTD.txt", sep="\t", header=None, names=["sentence", "label"])

In [72]:
astd_df

,sentence,label
0,بعد استقالة رئيس #المحكمة_الدستورية ننتظر استق...,OBJ
1,أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...,POS
2,البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...,NEG
3,#الحرية_والعدالة | شاهد الآن: #ليلة_الاتحادية ...,OBJ
4,الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقول...,NEUTRAL
...,...,...
9689,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,NEG
9690,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,NEG
9691,"#الشروق ""الداخلية"": 400 ألف مواطن تقدموا لأداء...",OBJ
9692,#هتحبك_لو صحتها من النوم علشان تقولها بحبك ;),POS


In [73]:
astd_df["label"].value_counts()

OBJ        6470
NEG        1642
NEUTRAL     805
POS         777
Name: label, dtype: int64

In [74]:
astd_df['label'].loc[(astd_df['label'] == "NEG")] = 0

In [75]:
astd_df['label'].loc[(astd_df['label'] == "NEUTRAL")] = 1

In [76]:
astd_df['label'].loc[(astd_df['label'] == "OBJ")] = 1

In [77]:
astd_df['label'].loc[(astd_df['label'] == "POS")] = 2

In [78]:
astd_df

,sentence,label
0,بعد استقالة رئيس #المحكمة_الدستورية ننتظر استق...,1
1,أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...,2
2,البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...,0
3,#الحرية_والعدالة | شاهد الآن: #ليلة_الاتحادية ...,1
4,الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقول...,1
...,...,...
9689,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,0
9690,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,0
9691,"#الشروق ""الداخلية"": 400 ألف مواطن تقدموا لأداء...",1
9692,#هتحبك_لو صحتها من النوم علشان تقولها بحبك ;),2


In [79]:
astd_df["label"].value_counts()

1    7275
0    1642
2     777
Name: label, dtype: int64

#### 2.8 Combining the datasets

In [80]:
df_list = [
    tead_pos_df,
    tead_neg_df,
    labr_df,
    arsas_df,
    ajgt_df,
    egypt_df,
    semeval_df,
    astd_df
]

data_df = pd.concat(df_list, axis=0, ignore_index=True)

In [81]:
data_df

,sentence,label
0,مراا حلو ان الاستاذه قايله برسل لكم التخطيط وت...,2
1,نلقى أي قروب يحكي علي ال ندير انسحاب تكتيكي لا...,2
2,يارب شعور إني أكون مروقه ومبسوطه يكون كل يوم,2
3,حاجات تزيد الانثي انوثه تتروش مع رجالها ولا تس...,2
4,وضع الانترنت واجد يضايق لا برودباند البيت ولا ...,2
...,...,...
693912,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,0
693913,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,0
693914,"#الشروق ""الداخلية"": 400 ألف مواطن تقدموا لأداء...",1
693915,#هتحبك_لو صحتها من النوم علشان تقولها بحبك ;),2


In [82]:
data_df["label"].value_counts()

2    462133
0    202725
1     29059
Name: label, dtype: int64

In [83]:
# Saving combined data
data_df.to_csv("data/sentiment-analysis/combined_arabic_data.csv", index=False)

In [84]:
# Testing loading combined data
data_df = pd.read_csv("data/sentiment-analysis/combined_arabic_data.csv")

In [85]:
data_df

,sentence,label
0,مراا حلو ان الاستاذه قايله برسل لكم التخطيط وت...,2
1,نلقى أي قروب يحكي علي ال ندير انسحاب تكتيكي لا...,2
2,يارب شعور إني أكون مروقه ومبسوطه يكون كل يوم,2
3,حاجات تزيد الانثي انوثه تتروش مع رجالها ولا تس...,2
4,وضع الانترنت واجد يضايق لا برودباند البيت ولا ...,2
...,...,...
693912,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,0
693913,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,0
693914,"#الشروق ""الداخلية"": 400 ألف مواطن تقدموا لأداء...",1
693915,#هتحبك_لو صحتها من النوم علشان تقولها بحبك ;),2


#### 2.9 Applying clean() function to all sentences in the data

In [86]:
print(data_df.iloc[693914]["sentence"])

#الشروق "الداخلية": 400 ألف مواطن تقدموا لأداء الحج وبدء إجراء القرعة 10 مايو المقبل


In [87]:
data_df["sentence"] = data_df["sentence"].map(clean)

In [88]:
data_df

,sentence,label
0,مراا حلو ان الاستاذه قايله برسل لكم التخطيط وت...,2
1,نلقى أي قروب يحكي علي ال ندير انسحاب تكتيكي لا...,2
2,يارب شعور إني أكون مروقه ومبسوطه يكون كل يوم,2
3,حاجات تزيد الانثي انوثه تتروش مع رجالها ولا تس...,2
4,وضع الانترنت واجد يضايق لا برودباند البيت ولا ...,2
...,...,...
693912,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,0
693913,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,0
693914,"""الداخلية"": 400 ألف مواطن تقدموا لأداء الحج و...",1
693915,صحتها من النوم علشان تقولها بحبك ;),2


In [89]:
print(data_df.iloc[693914]["sentence"])

 "الداخلية": 400 ألف مواطن تقدموا لأداء الحج وبدء إجراء القرعة 10 مايو المقبل


#### 2.10 Splitting data into a 80/20 train/test split

Train/val/test splits would be conducted in future experiments.

In [90]:
RANDOM_STATE = 420

train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=RANDOM_STATE)

In [91]:
train_df

,sentence,label
351637,من قبل يبدأ الموسم قايل طموحي هو الليغا فقط حت...,2
219680,القائمة المستدعاة لمواجهة أتلانتا مافي اعذار ا...,2
447570,يراضوني ب ورد ولا كلام حلو,0
213154,أسمعي معاي عبدالمجيد عبدالله خلي الوصل بيننا ا...,2
173996,للقلب الأسود هو الوحيد اللي حبه يزيد بقلبي لأن...,2
...,...,...
510579,ثاني شبيه لي بعد توأمي,0
586527,حينما تنتهى من قرائتها تجد حجم استفاداتك لا شئ,0
676415,لحد دلوقتي محدش جاوب صح عليا .,0
643462,صعب اوى انك تشوف دموع امكوانت مش عارف تعمل لي...,0


In [92]:
train_value_counts = train_df["label"].value_counts()
train_value_counts

2    369423
0    162431
1     23279
Name: label, dtype: int64

In [93]:
test_df

,sentence,label
510924,مسكوه الفرقه الفرقه محتاجه حد عارف قيمه الفريق...,0
25694,خاطرة رائعة عندما كنت طفلا كان والديك يخفون عن...,2
582693,"لغة صعبة الي حد ما,,,يميل فيها الكاتب الي بعض...",0
11318,انهارده يوم مهلك ابن حرام محاضرة وبعدين تمرين ...,2
515266,ابي رابط موقع نور بالسجل المدني,0
...,...,...
36708,عيشتك ملك لو تطلب انت روح هاك الروح فدوا لا تحزن,2
365433,مبروك الهادف يا ابا هادف بارك الله لكم في المو...,2
289997,أنا دخيل الله من نظرة عينها عجزت أوصف بالحلا م...,2
109661,لو الحب كان على قد المسافه انا احبك لآخر حدود ...,2


In [94]:
test_value_counts = test_df["label"].value_counts()
test_value_counts

2    92710
0    40294
1     5780
Name: label, dtype: int64

In [95]:
print("Ratios for train set:\n")
print(f"Negative ratio: {train_value_counts[0] / len(train_df)}")
print(f"Neutral ratio: {train_value_counts[1] / len(train_df)}")
print(f"Positive ratio: {train_value_counts[2] / len(train_df)}")

print("\nRatios for test set:\n")
print(f"Negative ratio: {test_value_counts[0] / len(test_df)}")
print(f"Neutral ratio: {test_value_counts[1] / len(test_df)}")
print(f"Positive ratio: {test_value_counts[2] / len(test_df)}")

Ratios for train set:

Negative ratio: 0.29259835030524217
Neutral ratio: 0.041934095072712306
Positive ratio: 0.6654675546220455

Ratios for test set:

Negative ratio: 0.29033606179386673
Neutral ratio: 0.04164745215586811
Positive ratio: 0.6680164860502652


#### 2.11 Calculating balanced class weights based on training set

In [96]:
np.unique(train_df["label"])

array([0, 1, 2])

In [97]:
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df["label"]),
    y=train_df["label"]
)

print(class_weights)

[1.13921809 7.9489812  0.50090095]


### 3- Preparing and training the Sentiment Classifier model from the CAMeL-Lab [bert-base-arabic-camelbert-mix](https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-mix) pre-trained language model

It is the language model with the __largest training corpus__ available by CAMeL-Lab. Composed of Modern Standard Arabic (MSA), dialectal Arabic (DA), and classical Arabic (CA).

* I have used this [tutorial](https://huggingface.co/docs/transformers/training) by huggingface as a reference how to do fine-tuned training for transformers using their 'transformers' library
* I have also used the fine-tuning code from CAMeL-Lab's GitHub [repository](https://github.com/CAMeL-Lab/CAMeLBERT) as a reference

#### 3.1 Importing base bert model

In [98]:
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-mix"

In [99]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassific

In [100]:
type(model)

transformers.models.bert.modeling_bert.BertForSequenceClassification

In [101]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [102]:
type(tokenizer)

transformers.models.bert.tokenization_bert_fast.BertTokenizerFast

In [103]:
input_test = tokenizer(clean("إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع ."))
input_test

{'input_ids': [2, 22718, 9276, 2247, 6448, 10182, 2401, 3913, 4644, 2488, 2687, 9710, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [104]:
tokenizer.decode(input_test["input_ids"])

'[CLS] إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع. [SEP]'

#### 3.2 Converting data from pandas dataframes to datasets.Dataset format

[Reference](https://huggingface.co/docs/datasets/loading#pandas-dataframe)

##### 3.2.1 Training data

In [105]:
train_dataset = Dataset.from_pandas(train_df)

In [106]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [107]:
train_dataset

Dataset({
    features: ['sentence', 'label', '__index_level_0__'],
    num_rows: 555133
})

In [108]:
train_dataset['sentence'][0]

'من قبل يبدأ الموسم قايل طموحي هو الليغا فقط حتى على حساب دوري الابطال الحمدلله هلا مدريد'

In [109]:
train_dataset['label'][0]

2

In [110]:
train_dataset['__index_level_0__'][0]

351637

##### 3.2.2 Test data

In [111]:
test_dataset = Dataset.from_pandas(test_df)

In [112]:
test_dataset

Dataset({
    features: ['sentence', 'label', '__index_level_0__'],
    num_rows: 138784
})

#### 3.3 Tokenizing the datasets

In [113]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

##### 3.3.1 Training set

In [114]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)

  0%|                                                                                                                                                                                                                                                             | 0/556 [00:00<?, ?ba/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 556/556 [00:12<00:00, 43.49ba/s]


In [115]:
tokenized_train_dataset

Dataset({
    features: ['sentence', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 555133
})

In [116]:
tokenized_train_dataset["sentence"][0]

'من قبل يبدأ الموسم قايل طموحي هو الليغا فقط حتى على حساب دوري الابطال الحمدلله هلا مدريد'

In [117]:
tokenized_train_dataset["label"][0]

2

In [118]:
tokenized_train_dataset["__index_level_0__"][0]

351637

In [119]:
tokenized_train_dataset["input_ids"][0]

[2,
 1908,
 2289,
 9880,
 4249,
 12698,
 2027,
 19599,
 1013,
 2139,
 23420,
 2574,
 2494,
 1961,
 3144,
 4299,
 12704,
 4062,
 9106,
 4879,
 3]

In [120]:
tokenized_train_dataset["token_type_ids"][0]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [121]:
tokenized_train_dataset["attention_mask"][0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

##### 3.3.2 Test set

In [122]:
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 139/139 [00:03<00:00, 42.50ba/s]


In [123]:
tokenized_test_dataset

Dataset({
    features: ['sentence', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 138784
})